# Restricted Two Body Problem: Elliptical Orbits Around a Central Mass

This is the general case of a Keperian orbit.<br>
A light body (e.g. a planet) orbits a heavy central body (e.g. the sun).  The orbit is an ellipse with the primary at one focus.

In [ ]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np

# Aliases
keras = tf.keras

In [33]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory, TimeHistory
from tf_utils import plot_loss_hist, EpochLoss, TimeHistory
from tf_utils import Identity

from orbital_element import OrbitalElementToConfig, ConfigToOrbitalElement

from r2b_data import make_traj_r2b, make_train_r2b, make_datasets_r2b, make_datasets_earth
from r2b import KineticEnergy_R2B, PotentialEnergy_R2B, AngularMomentum_R2B

In [3]:
# Lightweight serialization
# fname = '../data/r2b/r2bc.pickle'
# vartbl = load_vartbl(fname)

### Generate data sets and an example batch

In [4]:
# Generate one example trajectory
a = 1.0
e = 0.0
inc = 0.0
Omega = 0.0
omega = 0.0
f = 0.0
n_years = 2

inputs_traj, outputs_traj = make_traj_r2b(a=a, e=e, inc=inc, Omega=Omega, omega=omega, f=f, n_years=n_years)

In [5]:
inputs_traj.keys()

dict_keys(['t', 'q0', 'v0', 'mu'])

In [169]:
# Inputs for make_train_r2b
n_traj = 10
n_years = 2
a_min = 0.50
a_max = 32.0
e_max = 0.20
inc_max = np.pi/4.0
seed = 42

In [183]:
# Test make_train_r2b
inputs, outputs= make_train_r2b(n_traj=n_traj, n_years=n_years, a_min=a_min, a_max=a_max, 
                                e_max=e_max, inc_max=inc_max, seed=seed)

In [184]:
# Inputs for make_datasets_r2b
n_traj = 100
vt_split = 0.20
n_years = 2
a_min = 0.50
a_max = 32.0
e_max = 0.20
inc_max = np.pi/4.0
seed = 42
batch_size = 64

In [185]:
ds_trn, ds_val, ds_tst = make_datasets_r2b(n_traj=n_traj, vt_split=vt_split, n_years=n_years, a_min=a_min, a_max=a_max, 
                                e_max=e_max, inc_max=inc_max, seed=seed, batch_size=batch_size)

Unable to load data from ../data/r2b/1421569704.pickle.


In [186]:
# Create DataSet objects for toy size problem - earth orbits only (a=1, e=0)
ds_earth_trn, ds_earth_val, ds_earth_tst = make_datasets_earth(n_traj=n_traj, vt_split=vt_split, n_years=n_years)

Unable to load data from ../data/r2b/2367906283.pickle.


In [188]:
# Example batch
batch_in, batch_out = list(ds_earth_trn.take(1))[0]
print('Input field names: ', list(batch_in.keys()))
print('Output field names:', list(batch_out.keys()))

t = batch_in['t']
q0 = batch_in['q0']
v0 = batch_in['v0']
mu = batch_in['mu']

q = batch_out['q']
v = batch_out['v']
a = batch_out['a']
q0_rec = batch_out['q0_rec']
v0_rec = batch_out['v0_rec']
H = batch_out['H']
L = batch_out['L']

print(f'Example batch sizes:')
print(f't  = {t.shape}')
print(f'q0 = {q0.shape}')
print(f'v0 = {v0.shape}')
print(f'mu = {mu.shape}')

print(f'q  = {q.shape}')
print(f'v  = {v.shape}')
print(f'a  = {a.shape}')
# print(f'q0_rec = {q0_rec.shape}')
# print(f'v0_rec = {v0_rec.shape}')
print(f'H  = {H.shape}')
print(f'L  = {L.shape}')

Input field names:  ['t', 'q0', 'v0', 'mu']
Output field names: ['q', 'v', 'a', 'q0_rec', 'v0_rec', 'T', 'U', 'H', 'L']
Example batch sizes:
t  = (64, 731)
q0 = (64, 3)
v0 = (64, 3)
mu = (64,)
q  = (64, 731, 3)
v  = (64, 731, 3)
a  = (64, 731, 3)
H  = (64, 731)
L  = (64, 731, 3)


In [248]:
traj_size = 731

tf.debugging.assert_shapes(
    shapes = {
    # Inputs
    t: (batch_size, traj_size),
    q0: (batch_size, 3),
    v0: (batch_size, 3),
    mu: (batch_size,),
    # Outputs
    q: (batch_size, traj_size, 3),
    v: (batch_size, traj_size, 3),
    a: (batch_size, traj_size, 3),
    q0_rec: (batch_size, 3),
    v0_rec: (batch_size, 3),
    H: (batch_size, traj_size),
    L: (batch_size, traj_size, 3),
    })

**Call layers with physics computations**

In [190]:
T = KineticEnergy_R2B()(v)
T.shape

TensorShape([64, 731])

In [191]:
U = PotentialEnergy_R2B()([q, mu])
U.shape

TensorShape([64, 731])

In [192]:
L = AngularMomentum_R2B()([q, v])
L.shape

TensorShape([64, 731, 3])

**Conversion of initial configuration to orbital elements**

In [193]:
qx = q0[:,0]
qy = q0[:,1]
qz = q0[:,2]
vx = v0[:,0]
vy = v0[:,1]
vz = v0[:,2]
inputs_cart = (qx, qy, qz, vx, vy, vz, mu)

In [194]:
elts = ConfigToOrbitalElement()(inputs_cart)
a0, e0, inc0, Omega0, omega0, f0, M0, N0 = elts
mu0 = mu

In [195]:
# Review shapes
print(f'Example batch sizes:')
print(f'qx   = {qx.shape}')
print(f'vx   = {vx.shape}')
print(f'mu   = {mu.shape}')
print(f'a0   = {a0.shape}')
print(f'mu0  = {mu0.shape}')

Example batch sizes:
qx   = (64,)
vx   = (64,)
mu   = (64,)
a0   = (64,)
mu0  = (64,)


### Mathematical Model
**Compute position as a function of time from initial orbital elements**

In [301]:
def make_position_model_r2b_math(traj_size = 731):
    """
    Compute orbit positions for the restricted two body problem from 
    the initial orbital elements with a deterministic mathematical model.
    Factory function that returns a functional model.
    """
    # Create input layers
    t = keras.Input(shape=(traj_size), name='t')
    a0 = keras.Input(shape=(1,), name='a0')
    e0 = keras.Input(shape=(1,), name='e0')
    inc0 = keras.Input(shape=(1,), name='inc0')
    Omega0 = keras.Input(shape=(1,), name='Omega0')
    omega0 = keras.Input(shape=(1,), name='omega0')
    f0 = keras.Input(shape=(1,), name='f0')
    mu0 = keras.Input(shape=(1,), name='mu0')
    
    # The combined input layers
    inputs = [t, a0, e0, inc0, Omega0, omega0, f0, mu0]

    # Get batch size explicitly
    batch_size = t.shape[0]

    # Reshape inputs if necessary
    a0 = tf.reshape(a0, (-1, 1,))
    e0 = tf.reshape(e0, (-1, 1,))
    inc0 = tf.reshape(inc0, (-1, 1,))
    Omega0 = tf.reshape(Omega0, (-1, 1,))
    omega0 = tf.reshape(omega0, (-1, 1,))
    f0 = tf.reshape(f0, (-1, 1,))
    mu0 = tf.reshape(mu0, (-1, 1,))
    
    # Check inputs shapes
    tf.debugging.assert_shapes(shapes={
        a0: (batch_size, 1),
        e0: (batch_size, 1),
        inc0: (batch_size, 1),
        Omega0: (batch_size, 1),
        omega0: (batch_size, 1),
        mu0: (batch_size, 1),
    }, message='make_position_model_r2b_math / inputs')
    
    # Reshape t to (batch_size, traj_size, 1)
    # t_vec = keras.layers.Reshape(target_shape=(traj_size, 1), name='t_vec')(t)
    
    # Repeat the constant orbital elements to be vectors of shape (batch_size, traj_size)
    a = keras.layers.RepeatVector(n=traj_size, name='a')(a0)
    e = keras.layers.RepeatVector(n=traj_size, name='e')(e0)
    inc = keras.layers.RepeatVector(n=traj_size, name='inc')(inc0)
    Omega = keras.layers.RepeatVector(n=traj_size, name='Omega')(Omega0)
    omega = keras.layers.RepeatVector(n=traj_size, name='omega')(omega0)
    mu = keras.layers.RepeatVector(n=traj_size, name='mu')(mu0)

    # Check shapes
    tf.debugging.assert_shapes(shapes={
        a: (batch_size, traj_size, 1),
        e: (batch_size, traj_size, 1),
        inc: (batch_size, traj_size, 1),
        Omega: (batch_size, traj_size, 1),
        omega: (batch_size, traj_size, 1),
        mu: (batch_size, traj_size, 1),
    }, message='make_position_model_r2b_math / calcs')

    # Throwaway - duplicate the initial true anomaly
    # This is wrong, but the code should run
    f = keras.layers.RepeatVector(n=traj_size, name='f')(f0)
    
    tf.debugging.assert_shapes(shapes={
        f: (batch_size, traj_size, 1),
    }, message='make_position_model_r2b_math / f')
    
    # Wrap these up into one tuple of inputs
    inputs_elt = (a, e, inc, Omega, omega, f, mu,)
    
    # Convert from orbital elements to cartesian
    qx, qy, qz, vx, vy, vz = OrbitalElementToConfig(name='orbital_element_to_config')(inputs_elt)
    
    # Wrap up the outputs
    outputs = (qx, qy, qz)
    
    # Check shapes
    tf.debugging.assert_shapes(shapes={
        qx: (batch_size, traj_size, 1),
        qy: (batch_size, traj_size, 1),
        qz: (batch_size, traj_size, 1),
    }, message='make_position_model_r2b_math / outputs')
    
    # Wrap this into a model
    model = keras.Model(inputs=inputs, outputs=outputs, name='model_r2b_math')
    return model


In [302]:
position_model_math = make_position_model_r2b_math(traj_size)

In [303]:
print(f't     = {t.shape}')
print(f'a0    = {a0.shape}')
print(f'e0    = {e0.shape}')
print(f'inc0  = {inc0.shape}')
print(f'Omega0= {Omega0.shape}')
print(f'omega0= {omega0.shape}')
print(f'f0    = {f0.shape}')
print(f'mu0   = {mu0.shape}')

t     = (64, 731)
a0    = (64,)
e0    = (64,)
inc0  = (64,)
Omega0= (64,)
omega0= (64,)
f0    = (64,)
mu0   = (64,)


In [304]:
qx, qy, qz = position_model_math([t, a0, e0, inc0, Omega0, omega0, f0, mu])
print(f'qx = {qx.shape}')
print(f'qy = {qy.shape}')

qx = (64, 731, 1)
qy = (64, 731, 1)
